# MongoDB Handling

After installing the MongoDB server in your machine, you can use this notebook for handling the initial processes with the database.

Specifically, in this step, we utilize Python's `pymongo` library to exploit its capabilities for MongoDB server interaction.

**Important Note: Be sure that the MongoDB server is up and running as a service in the background.**

For example, in macOS, to run MongoDB (i.e. the mongod process) as a service, run:

* `brew services start mongodb-community`

To stop a mongod running as a macOS service, use the following command as needed:

* `brew services stop mongodb-community`

To install MongoDB in your system, follow the instructions here:

* https://www.mongodb.com/docs/manual/administration/install-community/


**Note:** You can modify any of the processes below, however, you have to explain your thoughts.

In [1]:
# import library for various processes with the OS
import os

## Load configuration

In [2]:
# import library for yaml handling
import yaml

In [3]:
config_path = os.path.join(os.getcwd(), "config.yml")

with open(config_path) as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

## MongoDB database instantiation

The relevant information for the MongoDB client connection, the database name, and collection name is located in the configuration file.

```
# DB Connection with the uri (host)
client: "mongodb://localhost:27017/"

# db name
db: "aiot_course"

# db collection
col: "NAME YOUR COLLECTION"
```

In [4]:
# import library for hanlding the MongoDB client
import pymongo
# import library for retrieving datetime
from datetime import datetime

### Create the database

To create a database in MongoDB, start by creating a MongoClient object, then specify a connection URL with the correct ip address and the name of the database you want to create.

MongoDB will create the database if it does not exist, and make a connection to it.

In [5]:
client = pymongo.MongoClient(config["client"])

In [6]:
db = client[config["db"]]

### Instantiate the collection

To create a collection in MongoDB, use database object and specify the name of the collection you want to create.

MongoDB will create the collection if it does not exist.

In [7]:
col = db[config["col"]]

Initially, no collection will be shown in MongoDB before you enter the first document!

## Create the data collection

Uploading the gathered data to MongoDB collection. The data directory structure should be as follows:

```
.
└── data/
    ├── class_A/
    │   ├── data_A_01.csv
    │   ├── data_A_02.csv
    │   └── ..
    ├── class_B/
    │   ├── data_B_01.csv
    │   ├── data_B_02.csv
    │   └── .
    └── class ...
```

In [8]:
# import library for hanlding the csv data and transformations
import pandas as pd
import json

Get data path:

In [9]:
data_path = os.path.join(os.getcwd(), "data")
print(data_path)

c:\Users\Bill\Desktop\IoT-Course-AIoT-project\data


List all files in a path:

In [10]:
classes_folders_list = [f for f in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, f))]
print(classes_folders_list)

['Clockwise_Circle', 'Counter_Clockwise_Circle', 'Down', 'Left', 'Right', 'Up', 'Wrist_turn_left', 'Wrist_turn_right']


In [11]:
# print files in folder
folder_path = os.path.join(data_path, classes_folders_list[2])
files_in_folder = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
print(files_in_folder)

['down_01.csv', 'down_02.csv', 'down_03.csv', 'down_04.csv', 'down_05.csv', 'down_06.csv', 'down_07.csv', 'down_08.csv', 'down_09.csv', 'down_10.csv']


Each document in the MongoDB database should have the following schema:

```json
{
  "data": {
    "acc_x": ["array", "of", "values"],
    "acc_y": ["array", "of", "values"],
    "acc_z": ["array", "of", "values"],
  },
  "label": "The label of the instance",
  "datetime": "MongoDB datetime object (it can be generated with the datetime.datetime.now() function"
}
```

Accordingly, if you are using gyroscope or both accelerometer and gyroscope, the following order and naming of the sensor keys should be defined:

* for gyroscope: `gyr_x`, `gyr_y`, `gyr_z` for the three axes
* for accelerometer and gyroscope: `acc_x`, `acc_y`, `acc_z`, `gyr_x`, `gyr_y`, `gyr_z` for the six axes

**Note: Be careful, the document is mandatory to have the aforementioned schema, in order to argue and proceed with the rest of the processes later on, in data engineering, plotting, etc.**

In [12]:
from utils import df_rebase

## Provide the code to upload the data to MongoDB

In [13]:
for class_folder in classes_folders_list:
    folder_path = os.path.join(data_path, class_folder)
    
    # Step 5: List all files in the class folder
    files_in_folder = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    print(f"Files in {class_folder}: {files_in_folder}")

    # Step 6: Loop through each file in the class folder
    for file in files_in_folder:
        if file.endswith('.csv'):
            file_path = os.path.join(folder_path, file)
            print(f"Processing file: {file_path}")
            
            # Step 7: Read the CSV file into a DataFrame
            df = pd.read_csv(file_path, delimiter=';')

            # Assuming the CSV has 'acc_x', 'acc_y', 'acc_z' columns
            acc_x = df['acc_x'].tolist()
            acc_y = df['acc_y'].tolist()
            acc_z = df['acc_z'].tolist()
            gyr_x = df['gyr_x'].tolist()
            gyr_y = df['gyr_y'].tolist()
            gyr_z = df['gyr_z'].tolist()


            # Step 8: Structure the data
            document = {
                "data": {
                    "acc_x": acc_x,
                    "acc_y": acc_y,
                    "acc_z": acc_z,
                    "gyr_x": gyr_x,
                    "gyr_y": gyr_y,
                    "gyr_z": gyr_z
                },
                "label": file,  # Use the class folder name as the label
                "datetime": datetime.now()  # Add the current timestamp
            }

            # Step 9: Insert the structured data into MongoDB
            col.insert_one(document)
            print(f"Inserted document for {file} into MongoDB")

# Close the MongoDB connection
client.close()

Files in Clockwise_Circle: ['clockwise_circle_01.csv', 'clockwise_circle_02.csv', 'clockwise_circle_03.csv', 'clockwise_circle_04.csv', 'clockwise_circle_05.csv', 'clockwise_circle_06.csv', 'clockwise_circle_07.csv', 'clockwise_circle_08.csv', 'clockwise_circle_09.csv', 'clockwise_circle_10.csv']
Processing file: c:\Users\Bill\Desktop\IoT-Course-AIoT-project\data\Clockwise_Circle\clockwise_circle_01.csv
Inserted document for clockwise_circle_01.csv into MongoDB
Processing file: c:\Users\Bill\Desktop\IoT-Course-AIoT-project\data\Clockwise_Circle\clockwise_circle_02.csv
Inserted document for clockwise_circle_02.csv into MongoDB
Processing file: c:\Users\Bill\Desktop\IoT-Course-AIoT-project\data\Clockwise_Circle\clockwise_circle_03.csv
Inserted document for clockwise_circle_03.csv into MongoDB
Processing file: c:\Users\Bill\Desktop\IoT-Course-AIoT-project\data\Clockwise_Circle\clockwise_circle_04.csv
Inserted document for clockwise_circle_04.csv into MongoDB
Processing file: c:\Users\Bill